This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
  
import json
with open("config_secret.json") as cnfg_file:
    cnfg = json.load(cnfg_file)
    
API_KEY = cnfg['api_key']

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import json
import math
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key': API_KEY, 'start_date': '2020-12-01', 'end_date': '2020-12-01'}
r = requests.get(url, params=params)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2020-12-01',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [['2020-12-01',
    112.2,
    112.2,
    111.5,
    112.0,
    None,
    51.0,
    5703.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
params = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'}
r = requests.get(url, params=params) # Task 1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
afx_x = r.json() # Task 2: Convert the returned JSON object into a Python dictionary.

In [6]:
max_high = float('-inf')
min_low = float('inf')
max_hl_change = float('-inf')
max_close = float('-inf')
min_close = float('inf')
tradvol_tot = 0
days = afx_x['dataset_data']['data']
tradvols = []

for day in days:
    high = day[2]
    low = day[3]
    hl_change = high - low
    close = day[4]
    tradvol = day[6]
 
    # Task 3: Calculate the highest opening price in this period.
    if high > max_high:
        max_high = high
           
    # Task 3: Calculate the lowest opening price in this period.
    if low < min_low:
        min_low = low
        
    # Task 4: The largest change in any one day (based on High and Low price)?
    if hl_change > max_hl_change:
        max_hl_change =  hl_change
        
    if close > max_close:
        max_close = close 
        
    if close < min_close:
        min_close = close 

    tradvol_tot = tradvol_tot + tradvol
    tradvols.append(tradvol)

max_close_change = max_close - min_close # Task 5: The largest change between any two days (based on Closing Price)
num_days = len(days)
tradvol_avg = tradvol_tot / num_days # Task 6: The average daily trading volume during this year

tradvols.sort()
# Task 7: The median trading volume during this year
if (num_days % 2) == 0:
    tradvol_median = tradvols[num_days/2]
else:
    tradvol_median = (tradvols[math.floor(num_days/2)] + tradvols[math.ceil(num_days/2)]) / 2

In [7]:
print('For the stock prices of Carl Zeiss Meditec in 2017:')
print('The highest opening price in 2017 was {}'.format(max_high))
print('The lowest closing price in 2017 was {}'.format(min_low))
print('The largest change between the low and high prices in any one day was {}'.format(round(max_hl_change, 2)))
print('The largest change between the closing prices on any two days was {}'.format(round(max_close_change, 2)))
print('The average daily trading volume during this year was {}'.format(round(tradvol_avg, 2)))
print('The median trading volume during this year was {}'.format(round(tradvol_median, 2)))

For the stock prices of Carl Zeiss Meditec in 2017:
The highest opening price in 2017 was 53.54
The lowest closing price in 2017 was 33.62
The largest change between the low and high prices in any one day was 2.81
The largest change between the closing prices on any two days was 19.03
The average daily trading volume during this year was 89124.34
The median trading volume during this year was 76443.0
